In [1]:
import sqlite3

Verbimustrite tabelite põhjal luuakse uus tabel **pattern_support**, kus on toodud verbimustrite esinemissagedused transaktsioonide andmebaasis. Tabelis on järgnevad veerud:

    pat_id - mustri ID
    verb_word - (pea)verb
    verb_compound - verbikonstruktsiooni muu osa
    phrase_case - kääne
    adp - kaassõna
    inf_verb - infiniitverb
    verb_occurrence_count - verbi(konstruktsiooni) esinemissagedus transaktsioonide andmebaasis
    absolute_support - mustri esinemissagedus transaktsioonide andmebaasis
    relative_support - mustri esinemissagedus võrreldes verbi esinemissagedusega transaktsioonide andmebaasis
    
  

In [6]:
con = sqlite3.connect("vp_data2.db")
cur = con.cursor()

In [7]:
cur.execute("""
DROP TABLE IF EXISTS pattern_support
""")

cur.execute("""
CREATE TABLE pattern_support AS
SELECT
    patterns_meta.pat_id,
    verb_word,
    verb_compound,
    phrase_case,
    adp,
    inf_verb,
    verb_match_count AS verb_occurrence_count,
    phrase_count AS absolute_support,
    CAST(phrase_count AS REAL) / CAST(verb_match_count AS REAL) * 100 AS relative_support
FROM
(
    SELECT 
        patterns.pat_id,
        verb_word,
        verb_compound,
        phrase_case,
        adp,
        inf_verb,
        count(*) AS verb_match_count
    FROM
        verb_matches
    INNER JOIN
        patterns
    ON
        patterns.pat_id = verb_matches.pat_id
    GROUP BY
        patterns.pat_id
) as tbl
INNER JOIN
    patterns_meta
ON
    tbl.pat_id = patterns_meta.pat_id
ORDER BY
    relative_support DESC
""")

In [8]:
con.close()